This code implements the market clearing model for one node with temporal shifting capacity. The node is associated with one data center that is able to delay part of its loads to the next time point. Assume the data center receives no delayed loads at the initial time point (t = 1) and delays no loads to later at the final time point (t = T).

In [1]:
using JuMP, Gurobi

In [50]:
scenario_num = 7;

In [51]:
# Node parameters
T = 5                       # Number of times
α_p = [10,20,10,15,20]      # bidding prices of supply
α_d = [30,60,40,50,45]      # bidding prices of demand
S = [50,50,50,50,50]        # supply capacity
D = [70,20,70,40,40]        # demand capacity
r = [20,20,20,20,20]        # reserved computing power

# Virtual link parameters
Del = [0,0,0,0];
if scenario_num == 2
    Del[1] = 10;
elseif scenario_num == 3
    Del[1] = 21;
elseif scenario_num == 4
    Del[1] = 21;
    Del[2] = 20;
elseif scenario_num == 5
    Del[1] = 21;
    Del[3] = 21;
elseif scenario_num == 6
    Del[1] = 21;
    Del[3] = 21;
    Del[4] = 10;
elseif scenario_num == 7
    Del = [100,100,100,100];
end

α_del = 3 * (copy(Del).>0);

# Ramping constraint parameters
R = 20;

In [52]:
# Optimizaiton formulation

m_v = Model(with_optimizer(Gurobi.Optimizer, OutputFlag=0))
@variable(m_v, p[1:T] >= 0)                         # supply
@variable(m_v, d[1:T] >= 0)                         # demand
@variable(m_v, δ[1:T+1] >= 0)                       # virtual shift

# Flow and virtual link constraints
for i in 1:T-1
    @constraint(m_v, δ[i+1] <= Del[i])              # virtual shift capacity constraints
end

@constraint(m_v, δ[1] == 0)
@constraint(m_v, δ[T+1] == 0)

# Supply and demand capacity constraints
@constraint(m_v, p .<= S)
@constraint(m_v, d .<= D)
for i in 1:T-1
    @constraint(m_v, d[i] + δ[i] - δ[i+1] >= 0)
    @constraint(m_v, d[i] + δ[i] - δ[i+1] <= D[i] + r[i])
end

# Ramping constraints
for i in 1:T-1
    @constraint(m_v, p[i+1] - p[i] <= R)
    @constraint(m_v, p[i+1] - p[i] >= -R)
end

# Node balance constraint
bals = Dict()
for i in 1:T
    bals[i] = @constraint(m_v, p[i] - d[i] - δ[i] + δ[i+1] == 0)
end

# Objecitve: social welfare
@objective(m_v, Min, sum(α_p.*p - α_d.*d) + sum(α_del.*δ[2:T]));


Academic license - for non-commercial use only


In [53]:
optimize!(m_v)
prices = zeros(T)
for i in 1:T
    prices[i] = dual(bals[i])
end

# Print info
println("The optimal value of virtual link model is ", -objective_value(m_v))
println("The nodal prices of virtual link model is ", prices)
println("Amt of cleared loads are ", value.(d))
println("The supplies are ", value.(p))
println("The virtual flows are ", value.(δ)[2:T])

Academic license - for non-commercial use only
The optimal value of virtual link model is 6200.0
The nodal prices of virtual link model is [23.0, 20.0, 26.0, 23.0, 20.0]
Amt of cleared loads are [70.0, 20.0, 70.0, 40.0, 40.0]
The supplies are [50.0, 40.0, 50.0, 50.0, 50.0]
The virtual flows are [20.0, 0.0, 20.0, 10.0]
